<a href="https://colab.research.google.com/github/davidfague/Neural-Modeling/blob/main/Soma_Axial_Currents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
is_colab = 'google.colab' in str(get_ipython())

In [ ]:
if is_colab:
    !git clone https://github.com/davidfague/Neural-Modeling.git
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/

In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as ss
from mpl_toolkits import mplot3d
import pdb #python debugger

In [9]:
output_folder = 'L5PCtemplate[0]_5.0Hz_196nseg_2000.0ms_108nbranch_29068NCs_29068nsyn'
v = h5py.File('./{}/Vm_report.h5'.format(output_folder),'r') #read segment voltage traces
segs = pd.read_csv('./{}/seg_info.csv'.format(output_folder)) # read segment values

In [15]:
print(segs.columns)

Index(['seg', 'seg_id', 'Beginning X Coord', 'Beginning Y Coord',
       'Beginning Z Coord', 'Center X Coord', 'Center Y Coord',
       'Center Z Coord', 'End X Coord', 'End Y Coord', 'End Z Coord',
       'seg diam', 'bmtk_id', 'x', 'sec', 'Type', 'Sec ID', 'sec diam',
       'sec nseg', 'Ra', 'seg_L', 'seg_SA', 'parent_seg_id', 'seg_elec_info',
       'netcons_per_seg', 'netcon_density_per_seg',
       'netcon_SA_density_per_seg', 'X', 'Elec_distanceQ'],
      dtype='object')


Probe class for making a segment's attributes accesible.

In [19]:
#Get Columns
segIDs=segs['seg index']
# segs = segs.set_index(['Type','Sec ID']).join(segs_degrees.set_index(['Type','Sec ID'])).reset_index() #not sure what this does
segs['sec index'] = segs['sec index'].astype(int)
segs['x'] = segs['x'].astype(float)
segs['Elec_distanceQ'] = 'None'
AllSegXCoord=segs['Center X Coord']
AllSegYCoord=segs['Center Y Coord']
AllSegZCoord=segs['Center Z Coord']
AllSegSec_ID=segs['sec index']
AllSegSec_name=segs['Type']
segLs=segs["seg L"]
segxs=segs["seg x"]
segdists=segs["seg h.distance"]
SegSecL=segs["sec"]
segdiams=segs["seg diam"]
SegSecnseg=segs["sec nseg"]
secRas=segs["section Ra"]
psegIDs=segs["pseg index"]
secids=segs["sec index"]
bmtkid=segs["BMTK ID"]
# sectypes=segs["Type"]

#get simulation time
seg_v=v['report']['biophysical']['data'][:,0] # get segment 0 voltage trace
print(len(seg_v)) #should be equivalnt to (tstop=1000ms)/(dt=0.1ms)=10000
tstop=1000 #simulation stop
dt=0.1 #timestep
t=np.arange(0,tstop,dt) #time array
print(len(t))
if len(t)!=len(seg_v):
  print('Update Time array')

psegids=[]
for i in range(len(psegIDs)):
  if np.isnan(psegIDs[i]) == False:
    psegids.append(int(psegIDs[i]))
  else:
    psegids.append(psegIDs[i])

KeyError: ignored

In [ ]:
probes=[]
class probe():
  def __init__(self,seg,sectionindex,sectionname,color,XCoord,YCoord,ZCoord,secRa,segdiam,segx,segdist,sectionL,sectionNseg,segL,seg_v,parentseg,sectype):
    self.seg=seg
    self.sectionindex=sectionindex
    self.sectionname=sectionname
    self.color=color
    self.XCoord=XCoord
    self.YCoord=YCoord
    self.ZCoord=ZCoord
    self.secRa=secRa
    self.segdiam=segdiam
    self.x=segx
    self.segdist=segdist
    self.secL=sectionL
    self.secnseg=sectionNseg
    self.segL=segL
    self.seg_v=seg_v
    self.name='sec:'+str(self.sectionname)+str(self.sectionindex)+' seg:'+str(self.seg)
    self.adjprobes=[]
    self.parentprobe=[]
    self.childprobes=[]
    self.axialcurrents=[]
    self.parentaxialcurrent=[]
    self.childrenaxialcurrents=[]
    self.halfsegRa=.01*self.secRa*(self.secL/2/self.secnseg)/(np.pi*(self.segdiam/2)**2)
    self.parentseg=parentseg
    self.sectype=sectype

    # try:
    #   self.axial_current=[Record_Axial_Current(section,single_seg=True)]
    # except:
    #   print(section)
for i in range(len(segIDs)):
  newprobe=probe(seg=i,sectionindex=AllSegSec_ID[i],sectionname=str(AllSegSec_name[i]),color='r',XCoord=AllSegXCoord[i],YCoord=AllSegYCoord[i],ZCoord=AllSegZCoord[i],
                secRa=secRas[i],segdiam=segdiams[i],segx=segxs[i],segdist=segdists[i],sectionL=SegSecL[i],sectionNseg=SegSecnseg[i],segL=segLs[i],
                seg_v=v['report']['biophysical']['data'][:,i],parentseg=psegids[i],sectype=sectypes[i])
  probes.append(newprobe)

Determine which segments are next to each other.

In [ ]:
#getting adjacent segments using parent segment IDs
#getting children segments
for i in range(len(psegids)): #iterate through segment index
  if np.isnan(psegids[i]) == False:
    for seg in segIDs: #check segIDs
      if psegids[i]==seg: #find parent seg from parent seg id
        probes[seg].adjprobes.append(probes[i]) #add child probe to probe's adjprobe list
        probes[seg].childprobes.append(probes[i]) #add child probe to probe's childprobe list
#getting parent segment
for i in range(len(segIDs)):
  if np.isnan(psegids[i]) == False:
    probes[i].adjprobes.append(probes[int(psegids[i])]) #add parent seg probe to adj probes
    probes[i].parentprobe.append(probes[int(psegids[i])]) #add parent seg probe to parent probes

Compute axial currents from each segment toward its adjacent segments.

In [ ]:
#compute axial currents between all segments
for probe in probes:
  for adjprobe in probe.adjprobes:
    axc=(probe.seg_v-adjprobe.seg_v)/(probe.halfsegRa+adjprobe.halfsegRa) #compute axial current using (v_in-v_out)/(halfsegRa+halfsegRa)
    probe.axialcurrents.append(axc)
    if [adjprobe] == probe.parentprobe:
      probe.parentaxialcurrent.append(axc)
    elif adjprobe in probe.childprobes:
      probe.childrenaxialcurrents.append(axc)

Find Segments adjacent to the soma segments

In [ ]:
#Find soma segments and adjacent segments
#Usefull subsetting method
# print(segs[(segs.Type=='soma')]) #how to separate by type
somasegIDs = segs[(segs.Type=='soma')]['segmentID'] #how to pull a column after separating by type
print(somasegIDs)
somaprobes=[]
for seg in somasegIDs:
  somaprobes.append(probes[seg])

adjsomaprobes=[]
for probe in somaprobes:
  for adjprobe in probe.adjprobes:
    if adjprobe.sectype != "soma": #need to make sure this works
      adjsomaprobes.append(adjprobe) # get probes adjacent to soma

somaaxialcurrents=[] #list of axial currents going into the soma if needed
for probe in adjsomaprobes:
    somaaxialcurrents.append(probe.parentaxialcurrent) # get axial current from outward segment to soma segment

Plot segments adjacent to soma

In [ ]:
plt.figure(figsize=(3,10))
ax = plt.plot(AllSegXCoord, AllSegYCoord,'.',color='k',markersize=6)
plt.vlines(110,400,500)
plt.text(0,450,'100 um')
plt.hlines(400,110,210)
plt.text(110,350,'100 um')
plt.xticks([])
plt.yticks([])
plt.box(False)

for probe in adjsomaprobes:
  plt.plot(AllSegXCoord[probe.seg],
         AllSegYCoord[probe.seg],
         '*',color=probe.color)

for probe in somaprobes:
  plt.plot(AllSegXCoord[probe.seg],
         AllSegYCoord[probe.seg],
         '*',color='blue')

Plot Axial Currents

In [ ]:
def plot_parent_axial_current(probe):
  title='Axial Current from ['+probe.name+'] to ['+probe.parentprobe[0].name+']'
  plt.figure(figsize=(12.8,4.8))
  plt.plot(t,probe.parentaxialcurrent[0],color=probe.color)
  plt.ylabel('nA')
  plt.title(title)
  plt.xlabel('time (ms)')
  plt.show()

In [ ]:
for probe in adjsomaprobes:
  plot_parent_axial_current(probe)